In [65]:
import geopandas as gpd
import pandas as pd 
from shapely.ops import unary_union
import osmnx as ox
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

CITIES = ['Lyon','Villeurbanne','Décines-Charpieu']

def group_adjacent_polygon(gdf):
    '''
    Créer une fonction qui regroupe les polygon adjacent pour en crée un seul. 
    Règle le problème où certaines géoémtrie comme les parcs sont subdivisé, 
    alors qu'on veut la géométrie du parc entier.
    '''
    gdf = gdf.reset_index()  # set index from 0 to n 
    gdf = gdf.drop(columns=['element_type'])
    # Identifier les polygones en contact
    contact_indices = []
    for i in range(len(gdf)): #pass through index 
        for j in range(i + 1, len(gdf)):
            if gdf.geometry.loc[i].touches(gdf.geometry.loc[j]):
                contact_indices.append(set([i,j]))

    # Regrouper l'ensemble des contact indices qui ont un élément en commmun : 
    grouped_geometry = {}
    for i,j in contact_indices:
        group_exists = False
        for k,grouped_geom in grouped_geometry.items():
            # Si i ou j est déjà dans un groupe, ajouter l'autre dans le groupe: 
            if (i in grouped_geom) or (j in grouped_geom):
                grouped_geometry[k] = list(set(grouped_geom)|set([i,j]))
                group_exists = True
                # sort de la boucle si trouvé 
                break
        # Si aucun groupe n'a été trouvé, on en crée un 
        if not group_exists:
            new_indice = len(grouped_geometry)
            grouped_geometry[new_indice] = [i,j]

    # Pour chaque grouped_geometry, les regrouper (gdf_to_add), et supprimer les geométrie non regroupé de gdf:
    gdf_to_add = gpd.GeoDataFrame()
    for k,grouped_geom in grouped_geometry.items():
        '''gdf.loc[grouped_geom].geometry'''
        geometries_to_merge = gdf.geometry[grouped_geom] 
        merged_geom = unary_union(geometries_to_merge)
        gdf_to_add_i = gpd.GeoDataFrame(dict(geometry = [merged_geom]),crs = gdf.crs)
        gdf_to_add_i['name'] = gdf.loc[grouped_geom[0]]['name']
        gdf_to_add_i['osmid'] = gdf.loc[grouped_geom[0]]['osmid']
        gdf_to_add = gpd.GeoDataFrame(pd.concat([gdf_to_add,gdf_to_add_i]),crs = gdf.crs)

        # Supprime les un-grouped geometry :
        gdf = gdf.drop(grouped_geom)

    new_gdf = gpd.GeoDataFrame(pd.concat([gdf,gdf_to_add]), crs=gdf.crs)
    return(new_gdf)

def get_leisure_from_cities(CITIES,tag_type,sub_tag,sports_tags = None):
    gdf = gpd.GeoDataFrame()
    for city in CITIES:
        place_name = f"{city}, France"
        tags = {tag_type: sub_tag}
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=DeprecationWarning)
            try : 
                gdf_i = ox.features_from_place(place_name, tags=tags)
                if sports_tags is not None:
                    gdf_i = gdf_i[gdf_i.sport.isin(sports_tags)]

                gdf_i = gdf_i[['name','geometry']]

                gdf_i = group_adjacent_polygon(gdf_i)
                gdf = gpd.GeoDataFrame(pd.concat([gdf,gdf_i]), crs=gdf_i.crs)
            except:
                print(f'No {tag_type}-{sub_tag} within {place_name}')
    return(gdf)

# Get POIs 'Leisure'

### Génère les POIs parcs : 
Rcupère les POIs de parc, groupe les sous-geometrie qui constitue un parc, et retourne uniquement les grands parcs:

In [66]:
tag_type = 'leisure'
sub_tag = 'park'
gdf_park = get_leisure_from_cities(CITIES,tag_type,sub_tag,sports_tags= None)
gdf_park = gdf_park.to_crs('epsg:2154')
gdf_park['area'] = gdf_park.geometry.area
gdf_park = gdf_park[gdf_park['area'] > 1e4]
gdf_park.head()


/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/

,osmid,name,geometry,area
3,4077207,Parc de la Tête d'Or,"POLYGON ((844429.070 6521700.071, 844427.874 6...",1.027388e+06
14,22664120,Parc de la Cerisaie,"POLYGON ((841225.210 6521089.897, 841230.896 6...",4.577566e+04
15,22727720,Parc Francis Popy,"POLYGON ((841683.113 6521404.516, 841682.834 6...",1.153397e+04
17,23269760,Parc Georges Bazin,"POLYGON ((846354.095 6518644.802, 846354.503 6...",2.803185e+04
22,24572931,Parc Chambovet,"POLYGON ((847355.478 6518139.166, 847358.636 6...",3.098740e+04


### Get POIs stadium : 

In [67]:
tag_type = 'leisure'
sub_tag = 'stadium'
sports_tags = ['rugby', 'football','soccer', 'basketball']
gdf_stadium = get_leisure_from_cities(CITIES,tag_type,sub_tag,sports_tags=sports_tags)
gdf_stadium

/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/

,osmid,name,geometry
0,4585967,Matmut Stadium Gerland,"POLYGON ((4.83278 45.72508, 4.83287 45.72507, ..."
0,85200814,Astroballe,"POLYGON ((4.90681 45.76680, 4.90699 45.76660, ..."
0,353267337,Groupama Stadium,"POLYGON ((4.98181 45.76665, 4.98209 45.76665, ..."


### Génère les POIs Centre commerciaux : 

In [53]:
leisure = 'park'
gdf_park = gdf_leisure[gdf_leisure.leisure == leisure][['name','geometry']]

gdf_park = group_adjacent_polygon(gdf_park)
gdf_park = gdf_park.to_crs('epsg:2154')
gdf_park['area'] = gdf_park.geometry.area
gdf_park = gdf_park[gdf_park['area'] > 1e4]
gdf_park.head()

/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/

In [2]:

# Save : 
if False: 
    save_path = f"{FOLDER_PATH}/POIs"
    print('save path : ',save_path)
    if not os.path.exists(save_path):
        os.mkdir(save_path)

    gdf_stadium['id'] = gdf_stadium['id'].apply(lambda x: int(x[1]))
    gdf_stadium.to_file(f"{save_path}/gdf_stadium.geojson", driver='GeoJSON')

# Load : 
if False:
    save_path = f"{FOLDER_PATH}/POIs"
    gpd.read_file(f"{save_path}/gdf_stadium.geojson")

gdf_stadium.explore()

In [8]:
gdf = ox.features_from_place(f"Lyon, France", tags={"amenity": True})

/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/

amenity         brand brand:wikidata  \
element_type osmid                                                
node         25214045         fuel  Total Access        Q154037   
             25468552  marketplace           NaN            NaN   
             25468556  marketplace           NaN            NaN   
             25733698  post_office           NaN            NaN   
             25733700   restaurant           NaN            NaN   

                             brand:wikipedia compressed_air fuel:diesel  \
element_type osmid                                                        
node         25214045  fr:Total (entreprise)            yes         yes   
             25468552                    NaN            NaN         NaN   
             25468556                    NaN            NaN         NaN   
             25733698                    NaN            NaN         NaN   
             25733700                    NaN            NaN         NaN   

                      fuel:e10 fuel:e85 fuel:lpg fuel:octane_98  ...  \
element_type osmid                                               ...   
node         25214045      yes      yes      yes            yes  ...   
             25468552      NaN      NaN      NaN            NaN  ...   
             25468556      NaN      NaN      NaN            NaN  ...   
             25733698      NaN      NaN      NaN            NaN  ...   
             25733700      NaN      NaN      NaN            NaN  ...   

                      est_min_height postal_code private building:material  \
element_type osmid                                                           
node         25214045            NaN         NaN     NaN               NaN   
             25468552            NaN         NaN     NaN               NaN   
             25468556            NaN         NaN     NaN               NaN   
             25733698            NaN         NaN     NaN               NaN   
             25733700            NaN         NaN     NaN               NaN   

                      ways max_level min_level prison:FR building:use  \
element_type osmid                                                      
node         25214045  NaN       NaN       NaN       NaN          NaN   
             25468552  NaN       NaN       NaN       NaN          NaN   
             25468556  NaN       NaN       NaN       NaN          NaN   
             25733698  NaN       NaN       NaN       NaN          NaN   
             25733700  NaN       NaN       NaN       NaN          NaN   

                      roof:material  
element_type osmid                   
node         25214045           NaN  
             25468552           NaN  
             25468556           NaN  
             25733698           NaN  
             25733700           NaN  

[5 rows x 778 columns]

In [49]:
gdf_nighclub

,amenity,name,geometry
0,nightclub,Fiesta Loca,POINT (4.82653 45.76729)
1,nightclub,Loft Club,POINT (4.84192 45.74851)
2,nightclub,Les Salons du NH,POINT (4.85894 45.73121)
3,nightclub,Vertige X,POINT (4.83621 45.76911)
4,nightclub,La Cour des Grands,POINT (4.81933 45.75378)
5,nightclub,it Bar,POINT (4.83580 45.77116)
6,nightclub,Le Fox Club Latino,POINT (4.84201 45.74826)
7,nightclub,Le Sucre,POINT (4.81501 45.73664)
8,nightclub,F&K Discoclub,POINT (4.85976 45.76644)
9,nightclub,Secret Dream,POINT (4.82887 45.74820)


In [35]:
[(name,amenity,osmid) for (name,amenity,osmid) in zip(gdf['name'],gdf['amenity'],gdf['osmid']) if (type(name) is str) and ('Nin' in name) ]

/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


[('Ninkasi - Sans Souci', 'pub', 61946256),
 ('Ninkasi Guillotière', 'bar', 80078846),
 ('Ninkasi - Saint-Paul', 'pub', 82677228),
 ('Ninkasi Gorge de Loup', 'pub', 2463494308),
 ('Ninkasi Vaise Industrie', 'bar', 2680176141),
 ('Ninkasi Cordeliers', 'pub', 4518436633),
 ('Ninie Cupcakes et Gourmandises', 'restaurant', 4775459269),
 ('Ninkasi', 'pub', 6185376347)]

In [48]:
gdf_filtered = gdf[#(gdf.amenity =='pub') |
    #(gdf.amenity =='bar') |
    (gdf.amenity =='nightclub') 
    #(gdf.amenity =='restaurant') |
    #(gdf.amenity =='music_venue')
    ][['amenity','name','geometry']]

# Add NinKasi Kao which doesn't existe here (closed since)
from shapely import Point 
ninkasi_kao = [4.830913047430858,45.72818916789741]
geom = Point(ninkasi_kao)

gdf_nighclub = gpd.GeoDataFrame(pd.concat([gdf_filtered, pd.DataFrame({'amenity': ['nightclub'],'name':['Ninkasi Kao'], 'geometry':[geom]})],ignore_index=True))

# Save:
if False :
    FOLDER_PATH = '../../../../data/rrochas/prediction_validation' 
    save_path = f"{FOLDER_PATH}/POIs"
    gdf_nighclub.to_file(f"{save_path}/gdf_nightclub.geojson", driver='GeoJSON')

# gdf_filtered.explore('amenity', marker_kwds=dict(radius=7)) 

/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/

In [12]:
#gdf[gdf.amenity =='restaurant'][['amenity','name','geometry']].explore() 

/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/

In [7]:
def find_amenities(amenity_type = "nightclub"):
    globals()[f"gdf_{amenity_type}"]  = gpd.GeoDataFrame()
    for city in ['Lyon','Villeurbanne','Décines-Charpieu']:
        place_name = f"{city}, France"
        tags = {"amenity": amenity_type}
        try: 
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", category=DeprecationWarning)
                filtered_pois = ox.features_from_place(place_name, tags=tags)
            #special_tags = ['rugby', 'football','soccer', 'basketball']
            #filtered_pois = pois[pois.sport.isin(special_tags)]
            gdf_i = gpd.GeoDataFrame({
                'id': filtered_pois.index,
                'nom': filtered_pois['name'],
                'geometry': filtered_pois['geometry']
            })
            globals()[f"gdf_{amenity_type}"] = gpd.GeoDataFrame(pd.concat([globals()[f"gdf_{amenity_type}"] ,gdf_i],ignore_index = True))
        except:
            print(f'No {amenity_type} in {city}')
    return globals()[f"gdf_{amenity_type}"]


#gdf_night_club =  find_amenities(amenity_type = "nightclub")
gdf_bars =  find_amenities(amenity_type = "bar")

#gdf_bars.explore()

# Leisire (Stadium) : 

In [115]:
gdf_leisure = ox.geometries_from_place(place_name, tags={"leisure": True})
leisure = gdf_leisure.leisure.unique()
# leisure = ['fitness_centre', 'escape_game', 'playground', 'sports_centre',
#       'picnic_table', 'slipway', 'pitch', 'sauna', 'hackerspace',
#       'amusement_arcade', 'marina', 'dance', 'fitness_station',
#       'bandstand', 'dog_park', 'garden', 'bowling_alley', 'park',
#       'sports_hall', 'playground_for_dogs', 'water_park', 'events',
#       'stadium', 'common', 'schoolyard', 'ice_rink', 'swimming_pool',
#       'track', 'outdoor_seating', 'bleachers', 'recreation_ground',
#       'miniature_golf', 'resort', 'grass', 'shrub']

/tmp/ipykernel_53774/1216858274.py:1: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf_leisure = ox.geometries_from_place(place_name, tags={"leisure": True})
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.

/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/

# Amenities : 

In [89]:
place_name = f"Lyon, France"

gdf_amenities = ox.geometries_from_place(place_name, tags={"amenity": True})
amenities = gdf_amenities.amenity.unique()

# amenities = ['fuel', 'marketplace', 'post_office', 'restaurant', 'post_box',
#       'cinema', 'pub', 'bicycle_rental', 'parking', 'school', 'library',
#       'pharmacy', 'fast_food', 'cafe', 'bank', 'bar', 'toilets',
#       'bicycle_parking', 'parking_entrance', 'police', 'nightclub',
#       'social_centre', 'drinking_water', .... ]
#

# == Set tags : 
# ___Bar / Boite: 
tags_bar=  ['bar;cafe;pub','bar','pub','nightclub']
# ___Theatre / Opera / 
tags_culture = ['theatre']
# ____Education : 
tags_school = ['university']
# ...

gdf_night_club =  gdf_amenities[gdf_amenities.amenity == 'nightclub'][['name','geometry']]
gdf_bar =  gdf_amenities[gdf_amenities.amenity == 'bar'][['name','geometry']]
gdf_culture =  gdf_amenities[gdf_amenities.amenity == 'cinema'][['name','geometry']]

#gdf_building = ox.geometries_from_place(place_name, tags={"building": True})
#buildings = gdf_building.amenity.unique()

/tmp/ipykernel_53774/1827981539.py:3: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf_amenities = ox.geometries_from_place(place_name, tags={"amenity": True})
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/fram

### Trian staiton in Lyon

In [124]:
place_name = f"Lyon, France"
gdf_train_station = ox.geometries_from_place(place_name, tags ={"building": "train_station"})
"""
osmid: 
Gare Part-dieu :  [196590753,62337529,1039333290,1191457432,1158815494]
Gare Perrache: [35143415]
Gare Vaise [31363448]
"""

part_dieu_ids = [196590753,62337529,1039333290,1191457432,1158815494]
gdf_train_station = gdf_train_station.reset_index()
gdf_train_station[gdf_train_station['osmid'].isin([part_dieu_ids])].explore()

/tmp/ipykernel_53774/1157927159.py:2: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf_train_station = ox.geometries_from_place(place_name, tags ={"building": "train_station"})
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages

In [24]:
place_name = f"Lyon, France"
ox.geometries_from_place(place_name, tags ={"railway": "station"})

/tmp/ipykernel_53774/2600919101.py:2: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  ox.geometries_from_place(place_name, tags ={"railway": "station"})
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: 

/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/

source  \
element_type osmid                                                           
node         4290857016                                                NaN   
             4290857018                                     SNCF - 06/2013   
             5365148732                                                NaN   
             5365152101                                                NaN   
             5365157959                                                NaN   
             5365165135                                                NaN   
             5365180176                                                NaN   
             5365180454                                                NaN   
             5365589507                                                NaN   
             5380086325                                                NaN   
             5380090834                                                NaN   
             5380092139                                                NaN   
             5385006337                                                NaN   
             5385022793                                                NaN   
             5385023672                                                NaN   
             5385025147                                                NaN   
             5385025382                                                NaN   
             5385025439                                                NaN   
             5385026358                                                NaN   
             5385029707                                                NaN   
             5385032105                                                NaN   
             5385036748                                                NaN   
             5385037483                                                NaN   
             5385052292                                                NaN   
             5387297446                                                NaN   
             5388249130                                                NaN   
             5509480923                                                NaN   
             5821110979                                                NaN   
             5821110983                                                NaN   
             8532840713                                                NaN   
             9469971777                                                NaN   
             9469971778                                                NaN   
             9924526864                                                NaN   
way          40702874                                                  NaN   
             319483070                                                 NaN   
             319836245                                                 NaN   
             332862090                                                 NaN   
             332925398                                                 NaN   
             336429489   smartdata Grand Lyon 2014 http://data.grandlyo...   
             336956496   smartdata Grand Lyon 2014 http://data.grandlyo...   
             674760183                                                 NaN   

                                                                  geometry  \
element_type osmid                                                           
node         4290857016                           POINT (4.86016 45.76057)   
             4290857018                           POINT (4.82533 45.74832)   
             5365148732                           POINT (4.83301 45.75736)   
             5365152101                           POINT (4.82596 45.76017)   
             5365157959                           POINT (4.83602 45.76786)   
             5365165135                           POINT (4.83096 45.72672)   
             5365180176                           POINT (4.80541 45.76638)   
           

In [23]:
place_name = f"Lyon, France"
ox.geometries_from_place(place_name, tags ={"railway": "subway"})

/tmp/ipykernel_53774/27575659.py:2: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  ox.geometries_from_place(place_name, tags ={"railway": "subway"})
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: Dep

/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/

geometry  \
element_type osmid                                                           
way          18617872    LINESTRING (4.82815 45.78184, 4.82824 45.78193...   
             51931058      LINESTRING (4.82911 45.78260, 4.82946 45.78285)   
             98713889    LINESTRING (4.82811 45.78193, 4.82794 45.78181...   
             98713892    LINESTRING (4.82786 45.78163, 4.82791 45.78171...   
             98713896    LINESTRING (4.82912 45.78268, 4.82935 45.78281...   
...                                                                    ...   
             850918881     LINESTRING (4.83589 45.76878, 4.83583 45.76903)   
             850918882   LINESTRING (4.83591 45.76894, 4.83599 45.76853...   
             850918883   LINESTRING (4.83162 45.77447, 4.83141 45.77445...   
             850918884   LINESTRING (4.83163 45.77453, 4.83186 45.77456...   
             1040609795    LINESTRING (4.83607 45.76793, 4.83602 45.76815)   

                            name network railway level  \
element_type osmid                                       
way          18617872    Ligne C     TCL  subway   NaN   
             51931058    Ligne C     TCL  subway   NaN   
             98713889        NaN     NaN  subway   NaN   
             98713892        NaN     NaN  subway   NaN   
             98713896        NaN     NaN  subway   NaN   
...                          ...     ...     ...   ...   
             850918881   Ligne C     TCL  subway    -2   
             850918882   Ligne C     TCL  subway    -2   
             850918883   Ligne C     TCL  subway   NaN   
             850918884   Ligne C     TCL  subway   NaN   
             1040609795  Ligne C     TCL  subway    -2   

                                                                     nodes  \
element_type osmid                                                           
way          18617872    [192064355, 1434432807, 638837977, 1434432837,...   
             51931058                              [475571014, 1434261013]   
             98713889     [1142015609, 1142015613, 1434432772, 1434432766]   
             98713892    [1142015596, 1434432780, 1434432785, 114201563...   
             98713896                 [2938051798, 1434258951, 1142015604]   
...                                                                    ...   
             850918881                            [7937874644, 3349897079]   
             850918882   [7937874639, 7937874642, 7937874640, 793787464...   
             850918883   [241595897, 10879656103, 987863556, 7937874624...   
             850918884   [7937874589, 7937874590, 7937874588, 108969054...   
             1040609795                           [3418368895, 3349897016]   

                          electrified                                 fixme  \
element_type osmid                                                            
way          18617872    contact_line  Cette ligne n'est pas à voie unique.   
             51931058    contact_line                                   NaN   
             98713889    contact_line                                   NaN   
             98713892    contact_line                                   NaN   
             98713896    contact_line                                   NaN   
...                               ...                                   ...   
             850918881   contact_line                                   NaN   
             850918882   contact_line                                   NaN   
             850918883   contact_line                                   NaN   
             850918884   contact_line                                   NaN   
             1040609795  contact_line                                   NaN   

                        frequency gauge  ... voltage bridge layer service  \
element_type osmid                       ...                                
way          18617872           0  1435  ...     750    NaN   NaN     NaN   
             51931058 